In [194]:
import os
import json
import numpy
import pandas
import ast
import re
import datetime
import pickle
import difflib

In [36]:
f = open("data/CrimeData/2011_01_assault_clustered.js", "r") 

In [37]:
lines = f.readlines()
f.close()

nHeaderLines = 3
nFooterLines = 3
records = len(lines) - nHeaderLines - nFooterLines


417

In [132]:
def parseLine(jsLine, crimeSet):
    cleanLine = jsLine.replace(" },\n", "")
    cleanLine = cleanLine.replace(" \"type\": \"Feature\", \"properties\": { \"Unnamed: 0\": ", "")
    cleanLine = cleanLine.split(",", 1)
    cleanLine = "{" + cleanLine[1]
    cleanLine = cleanLine.split(", \"geometry")
    cleanLine = cleanLine[0]
    cleanLine = re.sub("\"Offense 2\".*Location\"", "\"Location\"", cleanLine)
    #print(cleanLine)

    try:
        dictLine = ast.literal_eval(cleanLine)
        dictLine["DateTime"] = datetime.datetime.strptime(dictLine["Date"] + " " + dictLine["Time"], "%m\/%d\/%Y %I:%M %p")
        dictLine["CrimeSet"] = crimeSet
        return dictLine
    except ValueError:
        print("ERROR converting string")
        print(cleanLine)
        return "ERROR"


In [134]:
df = pandas.DataFrame()
path = "./data/CrimeData/"
allFiles = os.listdir(path)
nBadRecords = 0

for fileName in allFiles:
    print(fileName)
    f = open(path + fileName, "r") 
    lines = f.readlines()
    f.close()

    nHeaderLines = 3
    nFooterLines = 3
    crimeSet = fileName.split("_")[2]
    nRecords = len(lines) - nHeaderLines - nFooterLines
    print(nRecords)
    records = lines[nHeaderLines + 1:nHeaderLines + nRecords]
    iLine = nHeaderLines + 1
    for record in records:
        dictRecord = parseLine(record, crimeSet)
        if(dictRecord != "ERROR"):
            if(len(df.columns) == 0):
                df = pandas.DataFrame(dictRecord)
            else:
                df = df.append(dictRecord, ignore_index=True) 
        else:
            print("Error on line " + str(iLine))
            nBadRecords += 1;
        iLine += 1

2011_01_assault_clustered.js
411
2011_01_theft_clustered.js
713
2011_02_assault_clustered.js
375
2011_02_theft_clustered.js
603
2011_03_assault_clustered.js
466
2011_03_theft_clustered.js
935
2011_04_assault_clustered.js
438
2011_04_theft_clustered.js
942
2011_05_assault_clustered.js
483
2011_05_theft_clustered.js
922
2011_06_assault_clustered.js
467
2011_06_theft_clustered.js
876
2011_07_assault_clustered.js
423
2011_07_theft_clustered.js
953
2011_08_assault_clustered.js
410
2011_08_theft_clustered.js
805
2011_09_assault_clustered.js
357
2011_09_theft_clustered.js
848
2011_10_assault_clustered.js
364
2011_10_theft_clustered.js
960
2011_11_assault_clustered.js
382
2011_11_theft_clustered.js
840
2011_12_assault_clustered.js
391
2011_12_theft_clustered.js
800
2012_01_assault_clustered.js
438
2012_01_theft_clustered.js
697
2012_02_assault_clustered.js
413
2012_02_theft_clustered.js
680
2012_03_assault_clustered.js
464
2012_03_theft_clustered.js
816
2012_04_assault_clustered.js
458
2012_04

In [139]:
df.head()
pickleFile = open("df.pickle", "wb")
pickle.dump(df, pickleFile)
pickleFile.close()

In [243]:
df = pickle.load( open( "df.pickle", "rb" ))

In [245]:
df = df.drop(['Address,'], axis=1)
df.head()

,Address,CrimeSet,Date,DateTime,Incident Number,Location,Offense 1,Police District,Time,e_clusterK10,...,g_clusterK9,m_clusterK10,m_clusterK2,m_clusterK3,m_clusterK4,m_clusterK5,m_clusterK6,m_clusterK7,m_clusterK8,m_clusterK9
0,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,43.1093,SIMPLE ASSAULT,7.0,11:52 PM,2,...,6,2,1,0,0,0,1,4,0,3
1,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,-87.9528,SIMPLE ASSAULT,7.0,11:52 PM,2,...,6,2,1,0,0,0,1,4,0,3
2,2602 W PORT SUNLIGHT WA,assault,01\/30\/2011,2011-01-30 23:13:00,110300138,"[43.097656, -87.945886]",SIMPLE ASSAULT,5.0,11:13 PM,2,...,3,2,0,0,0,0,1,4,1,3
3,7403 N TEUTONIA AV,assault,01\/29\/2011,2011-01-29 02:37:00,110290012,"[43.152349, -87.958027]",SIMPLE ASSAULT,4.0,02:37 AM,5,...,6,5,1,2,0,0,1,4,0,6
4,5916 N 37TH ST,assault,01\/29\/2011,2011-01-29 02:08:00,110290015,"[43.125092, -87.958524]",SIMPLE ASSAULT,4.0,02:08 AM,5,...,6,5,1,2,0,0,1,4,0,6


In [146]:
## Get aldermanic districts
mprop = pandas.read_csv("./data/Mprop.csv")
mprop.columns

C:\Users\marie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17,26,43,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['TAXKEY', 'SUB_ACCT', 'YR_ASSMT', 'CHK_DIGIT', 'TAX_RATE_CD',
       'PLAT_PAGE', 'HOUSE_NR_LO', 'HOUSE_NR_HI', 'HOUSE_NR_SFX', 'SDIR',
       'STREET', 'STTYPE', 'C_A_CLASS', 'C_A_SYMBOL', 'C_A_LAND', 'C_A_IMPRV',
       'C_A_TOTAL', 'C_A_EXM_TYPE', 'C_A_EXM_LAND', 'C_A_EXM_IMPRV',
       'C_A_EXM_TOTAL', 'P_A_CLASS', 'P_A_SYMBOL', 'P_A_LAND', 'P_A_IMPRV',
       'P_A_TOTAL', 'P_A_EXM_TYPE', 'P_A_EXM_LAND', 'P_A_EXM_IMPRV',
       'P_A_EXM_TOTAL', 'LAST_VALUE_CHG', 'REASON_FOR_CHG', 'CONVEY_DATE',
       'CONVEY_TYPE', 'CONVEY_FEE', 'CHG_NR', 'DIV_ORG', 'DIV_DROP',
       'OWNER_NAME_1', 'OWNER_NAME_2', 'OWNER_NAME_3', 'OWNER_MAIL_ADDR',
       'OWNER_CITY_STATE', 'OWNER_ZIP', 'LAST_NAME_CHG', 'NEIGHBORHOOD',
       'EXM_ACREAGE', 'EXM_PER_CT_LAND', 'EXM_PER_CT_IMPRV', 'BLDG_TYPE',
       'NR_STORIES', 'BASEMENT', 'ATTIC', 'NR_UNITS', 'BLDG_AREA', 'YR_BUILT',
       'SWIM_POOL', 'FIREPLACE', 'AIR_CONDITIONING', 'NR_ROOMS', 'BEDROOMS',
       'BATHS', 'POWDER_ROOMS', 'PARKING_SP

In [192]:
mprop["HOUSE_NR_SFX"] = mprop["HOUSE_NR_SFX"].fillna('')
mprop.loc[mprop["HOUSE_NR_SFX"].str.isalnum(), "HOUSE_NR_SFX"] = "-" + mprop["HOUSE_NR_SFX"]

mprop["FULL_ADDRESS"] = mprop["HOUSE_NR_LO"].map(str) + mprop["HOUSE_NR_SFX"] + " " + mprop["SDIR"] + " " + mprop["STREET"] + " " + mprop["STTYPE"]
mprop.head()

print(mprop['FULL_ADDRESS'].isnull().sum())
mprop['FULL_ADDRESS'] = mprop['FULL_ADDRESS'].fillna('')
print(mprop['FULL_ADDRESS'].isnull().sum())

,TAXKEY,SUB_ACCT,YR_ASSMT,CHK_DIGIT,TAX_RATE_CD,PLAT_PAGE,HOUSE_NR_LO,HOUSE_NR_HI,HOUSE_NR_SFX,SDIR,...,ELEMENT,MSLINK,MAPID,TAX_DELQ,BI_VIOL,RAZE_STATUS,MAP_NAME,MAP_EXT,Unnamed: 92,FULL_ADDRESS
0,19995111,0,2018,4,1,102,9202,9202,,N,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,9202 N 124TH ST
1,19996100,0,2018,4,1,102,12125,12125,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,12125 W COUNTY LINE RD
2,19996210,0,2018,8,1,104,12301,12301,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,12301 W COUNTY LINE RD
3,19997000,0,2018,3,1,101,11715,11715,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,11715 W COUNTY LINE RD
4,19998200,0,2018,1,1,101,11617,11617,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,11617 W COUNTY LINE RD


In [246]:
#df = df.drop(["Adermanic_District"], axis=1)
df["Aldermanic_District"] = numpy.nan
df.head()
totalFound = 0
totalFuzzy = 0
totalNotFound = 0
maxIndex = 0

,Address,CrimeSet,Date,DateTime,Incident Number,Location,Offense 1,Police District,Time,e_clusterK10,...,m_clusterK10,m_clusterK2,m_clusterK3,m_clusterK4,m_clusterK5,m_clusterK6,m_clusterK7,m_clusterK8,m_clusterK9,Aldermanic_District
0,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,43.1093,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,NaN
1,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,-87.9528,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,NaN
2,2602 W PORT SUNLIGHT WA,assault,01\/30\/2011,2011-01-30 23:13:00,110300138,"[43.097656, -87.945886]",SIMPLE ASSAULT,5.0,11:13 PM,2,...,2,0,0,0,0,1,4,1,3,NaN
3,7403 N TEUTONIA AV,assault,01\/29\/2011,2011-01-29 02:37:00,110290012,"[43.152349, -87.958027]",SIMPLE ASSAULT,4.0,02:37 AM,5,...,5,1,2,0,0,1,4,0,6,NaN
4,5916 N 37TH ST,assault,01\/29\/2011,2011-01-29 02:08:00,110290015,"[43.125092, -87.958524]",SIMPLE ASSAULT,4.0,02:08 AM,5,...,5,1,2,0,0,1,4,0,6,NaN


In [ ]:

for index, row in df.iterrows():
    if index > maxIndex:
        if(index % 100 == 0 and index != 0):
            print("INDEX: " + str(index))
            print("found: " + str(totalFound))
            print("fuzzy: " + str(totalFuzzy))
            print("not found: " + str(totalNotFound))
            print("Null dists: " + str(df["Aldermanic_District"].isnull().sum()))
            pickleFile = open("alder.pickle", "wb")
            pickle.dump(df, pickleFile)
            pickleFile.close()

        rootAddr = row['Address'].split(" #")[0];
        found = mprop['FULL_ADDRESS'].str.contains(rootAddr).any()
        if found == True:
            totalFound += 1
            dist = mprop[mprop['FULL_ADDRESS']==rootAddr]['GEO_ALDER'].tolist()
            if len(dist) == 0:
                dist = numpy.nan
            else:
                dist = dist[0]
            df.loc[index, 'Aldermanic_District'] = dist
        else:
            print("    " + rootAddr)
            matches = difflib.get_close_matches(rootAddr, mprop['FULL_ADDRESS'].tolist(), 5)
            #mprop.loc[mprop["FULL_ADDRESS"] == matches, "GEO_ALDER"]
            dist = mprop[mprop["FULL_ADDRESS"].isin(matches)]["GEO_ALDER"].mode()
            #row['Aldermanic_District'] = dist[0]
            #df.iloc[index, 'Aldermanic_District'] = dist[0]
            df.loc[index, 'Aldermanic_District'] = dist[0]
            totalFuzzy += 1
        maxIndex = index

    115 N 68TH ST
    538 S 74TH ST
    6217-A W BURLEIGH ST
    2674 N 64TH ST
    5131 W NORTH AV
INDEX: 300
found: 17
fuzzy: 5
not found: 0
Null dists: 71071
    1849 W GREENFIELD AV
    1418 S CESAR E CHAVEZ DR
    1408 W MADISON ST
    1515 W LAPHAM BL
    2015 S 16TH ST
    1650 S 6TH ST
    1571 S 15TH ST
    1616 W BECHER ST
    1550-B W CLEVELAND AV
    1731 W MITCHELL ST
    2401 S 15TH PL
    2406 S 14TH ST
    700 W MADISON ST
    1427 S 11TH ST
    2248-A S 15TH ST
    1802-B S 19TH ST
    3464 S 24TH ST
    2872-A S 16TH ST
    2475 S 5TH PL
    2829 S 9TH PL
    4000 W GALENA ST
    2712 N 12TH ST
    1918 W WRIGHT ST
    2648 N 49TH ST
    2326 N 51ST ST
    3604 W LINDEN PL
    4950 W LISBON AV
    2173 N 35TH ST
    2911 W NORTH AV
    1440 N 39TH ST
    2212 N 24TH PL
    2553 N 20TH ST
    1826 W GALENA ST
    2673 N 41ST ST
    2107 N 29TH ST
    1802 N 36TH ST
    3406 W FLORIST AV
    4578 N TEUTONIA AV
    5027 N 35TH ST
    1800 W WELLS ST
    624-I N 32ND ST
 

    7614 W WABASH CT
    5623 W LISBON AV
    3506 W MT VERNON AV
    2756 N 56TH ST
    5223 W MEINECKE AV
    504-A S 76TH ST
    409-A S 69TH ST
    7181 W BECKETT AV
    3810 S 48TH ST
INDEX: 1400
found: 759
fuzzy: 363
not found: 0
Null dists: 69985
    602 S 2ND ST
    1022 W ORCHARD ST
    627-A S 6TH ST
    1200 W LAPHAM BL
    1943-A S 5TH PL
    2339 S 7TH ST
    1310 W LINCOLN AV
    2434-B S 9TH ST
    1559-A S 9TH ST
    2466-B S 9TH ST
    2526 W ABBOTT AV
    3734-A S 14TH ST
    2306 W MORGAN AV
    2685-B S 9TH PL
    2506 S 9TH PL
    2606 N 48TH ST
    2406 N 49TH ST
    2765-A N 22ND ST
    2527 N 22ND ST
    1500 W MEINECKE AV
    3501 W VLIET ST
    2149 N 41ST ST
    2636 N 24TH ST
    2632 N 22ND ST
    1201 N 35TH ST
    3130 W NORTH AV
    2471 N 34TH ST
    2425-A N 39TH ST
    4812 W LISBON AV
    4109 W NORTH AV
    2248 N 30TH ST
    4810 W LLOYD ST
INDEX: 1500
found: 827
fuzzy: 395
not found: 0
Null dists: 69887
    3818 W HAMPTON AV
    2027 W ATKINSON AV

    771 W WINDLAKE AV
    818 S WATER ST
    1964 S CONGO AV
    2458 S 17TH ST
    3032 W IONA TR
INDEX: 2500
found: 1507
fuzzy: 715
not found: 0
Null dists: 68908
    3547 S 5TH ST
    574 W ARTHUR AV
    2603 S 6TH ST
    2617 N 49TH ST
    2702 W CENTER ST
    2823 N 8TH ST
    2577 N 10TH ST
    2109 N 41ST ST
    3513 W LISBON AV
    2506 N 39TH ST
    2400 N 27TH ST
    2400 W FOND DU LAC AV
    2838 N 35TH ST
    3704 W GARFIELD AV
    2350 N 51ST ST
    1201 N 35TH ST
    1802 N 36TH ST
    1802 N 36TH ST
    2457 N 9TH ST
    2556 N 18TH ST
    5966 N 35TH ST
    5100 W MILL RD
    5463-A N 35TH ST
    7200 N TEUTONIA AV
    4071 N 12TH ST
    4071 N 12TH ST
    5966 N 41ST ST
    4700 N TEUTONIA AV
    5000 W HAMPTON AV
    2151 W ATKINSON AV
    4231 W REICHERT PL
INDEX: 2600
found: 1576
fuzzy: 746
not found: 0
Null dists: 68808
    2112 W FAIRMOUNT AV
    4371 N 29TH ST
    5134 N 40TH ST
    5363 N 38TH ST
    3500 W HAMPTON AV
    4476 N 29TH ST
    6020 N 35TH ST
    20

    7600 W APPLETON AV
    4071 N 68TH ST
    7123 W SILVER SPRING DR
    4054 N 54TH ST
    5200 W CAPITOL DR
    5969 N 68TH ST
    7520 W CONGRESS ST
    8548 W APPLETON AV
    5910 N 76TH ST
    6313-A N 84TH ST
    5496 N 72ND ST
    5451 N 64TH ST
    5585 N 74TH ST
    5636 N 95TH ST
    4131 N 56TH ST
    7129 W SILVER SPRING DR
    2220 N FARWELL AV
INDEX: 3600
found: 2235
fuzzy: 1087
not found: 0
Null dists: 67827
    821 E BURLEIGH ST
    1350 N 35TH ST
    2414 W WELLS ST
    625 N MILWAUKEE ST
    3000 W CLYBOURN ST
    3225 W WISCONSIN AV
    1235 W JUNEAU AV
    450 N PLANKINTON AV
    1601 N 25TH ST
    1015 N 10TH ST
    1325 N 24TH PL
    10207 W KIEHNAU AV
    9800 W LEON TR
    9909 W FOND DU LAC AV
    9923 W FOND DU LAC AV
    8856 W FAIRMOUNT AV
    2950 N 5TH ST
    700 W LOCUST ST
    2954-A N 19TH ST
    3537-A N 1ST ST
    2826 N DR MARTIN LUTHER KING JR DR
    3309-A N 12TH ST
    2519-A N BUFFUM ST
    140 E BURLEIGH ST
    3575 N 11TH ST
    3292 N 23RD ST

    3873 N 76TH ST
    5921 N LOVERS LANE RD
    7600 W CENTER ST
    9100 W FOND DU LAC AV
    8228 W CONGRESS ST
    2100 N 6TH ST
    3605 N MARTIN L KING JR DR
    3252 N 3RD ST
    2231 N HUMBOLDT AV
    1400 W ATKINSON AV
    3286 N 14TH ST
    2231 N HUMBOLDT AV
    200 E CAPITOL DR
    2778 N MARTIN L KING JR DR
    1500 N 6TH ST
    2423 N PALMER ST
    3156 N 11TH ST
    600 W VIENNA AV
    1411 N 7TH ST
    2260 N HUMBOLDT AV
    1407 N MARTIN L KING JR DR
    3023 N PALMER ST
    500 E KEEFE AV
    3225 N 15TH ST
    1549 N 9TH ST
    3960 N 24TH ST
INDEX: 4700
found: 2966
fuzzy: 1456
not found: 0
Null dists: 66757
    4216 N 44TH WA
    5501 W HAMPTON AV
    3040 W HOPKINS ST
    3618 N 58TH BL
    2810 N 24TH ST
    2804 N 45TH ST
    2915 N 45TH ST
    4300 W FOND DU LAC AV
    3300 W AUER AV
    4953 N 53RD ST
    6000 W CONGRESS ST
    5000 W FOND DU LAC AV
    3812 N 37TH ST
    4300 N SHERMAN BL
    3329 N 35TH ST
    4620 W OLIVE WA
    2625 W TOWNSEND ST
    2847 N

    2432 N CRAMER ST
    1533 E ROYALL PL
    600-BLO E CENTER ST
    2519 N PROSPECT AV
    1500 E PARK PL
    3166-A N PIERCE ST
    1710 N ARLINGTON PL
INDEX: 5800
found: 3725
fuzzy: 1797
not found: 0
Null dists: 65665
    1238 E BRADY ST
    2963 N FRATNEY ST
    2605 N FRATNEY ST
    544 E OGDEN AV
    2015 E WOODSTOCK PL
    1255 N MARSHALL ST
    1327 E ALBION ST
    1340 N MARSHALL ST
    2351 N NEWHALL ST
    1530 N JEFFERSON ST
    524 E LYON ST
    2577 N OAKLAND AV
    1425 N CASS ST
    544 E OGDEN AV
    1301 N BROADWAY
    1818-B N PULASKI ST
    1425 N CASS ST
    1930 E LOCUST ST
    1310 N MARSHALL ST
    1425 N CASS ST
    631 E HADLEY ST
    3226 N HUMBOLDT BL
    1337 E BRADY ST
    618 N WATER ST
    500 E WELLS ST
    1010 N LINCOLN MEMORIAL DR
    346 N BROADWAY
    1015 N 10TH ST
    1900 N SUMMIT AV
    755 N 10TH ST
    724 N OLD WORLD THIRD ST
    700 N 9TH ST
    1215 N 9TH ST
    649 E ERIE ST
    114 N JACKSON ST
INDEX: 5900
found: 3790
fuzzy: 1832
not fo

    3805 S MINER ST
    150 E NATIONAL AV
    902 W NATIONAL AV
    1903 S 19TH ST
    1135 S CESAR E CHAVEZ DR
    133 W OREGON ST
    700 W HISTORIC MITCHELL ST
    316 E WALKER ST
    212 S BARCLAY ST
    1711 S 9TH ST
    700 W SCOTT ST
    813 S 1ST ST
    500 W WALKER ST
INDEX: 6900
found: 4460
fuzzy: 2162
not found: 0
Null dists: 64586
    700 W VIRGINIA ST
    133 W OREGON ST
    2024 S 17TH ST
    818 S WATER ST
    900 W PIERCE ST
    1643 S 2ND ST
    1414 W SCOTT ST
    400 W VIRGINIA ST
    1315 W MADISON ST
    2213 W LAYTON AV
    807 W LAYTON AV
    2814 S 15TH PL
    2114 W COLLEGE AV
    3730 S 13TH ST
    4351 S 20TH ST
    5141 S 13TH ST
    2230 S 17TH ST
    5054 S 14TH ST
    3801 S 6TH ST
    3801 S 6TH ST
    2435 S 19TH ST
    1300 W HOWARD AV
    3649 S 13TH ST
    565 W LAYTON AV
    3716 S 27TH ST
    2810 S 14TH ST
    250 W HOLT AV
    2745 S LOGAN AV
    2420 S LINCOLN MEMORIAL DR
    2400 S KINNICKINNIC AV
INDEX: 7000
found: 4530
fuzzy: 2192
not found: 

    5229 N SHERMAN BL
    8245 N 107TH ST
    9172 N 86TH CT
    8526 N SERVITE DR
    9628 W BROWN DEER RD
    7739 N MARINERS ST
    115 N 68TH ST
    2837 N 58TH ST
    6645 W MOLTKE AV
    5709 W NORTH AV
    7802 W WATERFORD AV
    1844 W GREENFIELD AV
    1645 W FOREST HOME AV
    1238 S 21ST ST
    1704 S 7TH ST
    1149 W WINDLAKE AV
    1649 S 5TH ST
    2113 S 11TH ST
    1546 S 7TH ST
    2032 S 17TH ST
    370 E ROSEDALE AV
    2690-B S 10TH ST
    2300 S KINNICKINNIC AV
    4221 W NORTH AV
    4950 W LISBON AV
    2437 N 40TH ST
INDEX: 8000
found: 5192
fuzzy: 2530
not found: 0
Null dists: 63504
    2457 N 39TH ST
    2415 N 40TH ST
    2851 N 33RD ST
    2815-A N 28TH ST
    2231 N 44TH ST
    4600 W NORTH AV
    1703 N 17TH ST
    2757 N 33RD ST
    2644 N 24TH ST
    2578 N 49TH ST
    1908 N 27TH ST
    1951 N 33RD ST
    2137 N 48TH ST
    2762 N 18TH ST
    4850 W CLARKE ST
    2500 N 37TH ST
    3500 W HAMPTON AV
    4934 N 24TH ST
    4852 N 22ND ST
    4954 N 37TH 

In [228]:
a = "7061 N TEUTONIA AV #206"
poss = mprop['FULL_ADDRESS'].tolist();
matches = difflib.get_close_matches(a, poss, 5)
print(matches)
dist = mprop[mprop["FULL_ADDRESS"].isin(matches)]["GEO_ALDER"].mode()
dist[0]

['7061 N TEUTONIA AV', '7501 N TEUTONIA AV', '7026 N TEUTONIA AV', '7017 N TEUTONIA AV', '3706 N TEUTONIA AV']


1.0

In [287]:
df.head()

,Address,CrimeSet,Date,DateTime,Incident Number,Location,Offense 1,Police District,Time,e_clusterK10,...,m_clusterK10,m_clusterK2,m_clusterK3,m_clusterK4,m_clusterK5,m_clusterK6,m_clusterK7,m_clusterK8,m_clusterK9,Aldermanic_District
0,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,43.1093,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,1.0
1,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,-87.9528,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,14.0
2,2602 W PORT SUNLIGHT WA,assault,01\/30\/2011,2011-01-30 23:13:00,110300138,"[43.097656, -87.945886]",SIMPLE ASSAULT,5.0,11:13 PM,2,...,2,0,0,0,0,1,4,1,3,1.0
3,7403 N TEUTONIA AV,assault,01\/29\/2011,2011-01-29 02:37:00,110290012,"[43.152349, -87.958027]",SIMPLE ASSAULT,4.0,02:37 AM,5,...,5,1,2,0,0,1,4,0,6,1.0
4,5916 N 37TH ST,assault,01\/29\/2011,2011-01-29 02:08:00,110290015,"[43.125092, -87.958524]",SIMPLE ASSAULT,4.0,02:08 AM,5,...,5,1,2,0,0,1,4,0,6,1.0


In [299]:
maxIndex
df["Aldermanic_District"].isnull().sum()

71092